In [2]:
from prepare_data.utils import csv2df, clean_dataset
from prepare_data.const_predprice import  TARGET_COLS, BASE_COLS, BATCH_SIZE, DEPTH, PREDICT_LAG
from prepare_data.constants import GLASS_COLS
import numpy as np # библиотека нампи

# make train data

In [3]:
!python prepare_data\predprice_traindata.py

In [4]:
x_train_sc = np.load("./predprice/y_train_sc.npy")
x_val_sc = np.load("./predprice/x_val_sc.npy")
y_train_sc = np.load("./predprice/y_train_sc.npy")
y_val_sc = np.load("./predprice/y_val_sc.npy")

In [9]:
import tensorflow as tf           # библиотека машинного обучения
# Для генерации выборки временных рядов
TSG = tf.keras.preprocessing.sequence.TimeseriesGenerator

In [10]:
# Создаем генератор для обучения
train_datagen = TSG(x_train_sc, y_train_sc,          # В качестве параметров наши выборки
                    length = DEPTH,            # Анализируем по 21 прошедшим точкам
                    sampling_rate = 1,         # Для каждой точки
                    batch_size = BATCH_SIZE)   # Размер batch, который будем скармливать модели

# Создаем аналогичный генератор для валидации при обучении
val_datagen = TSG(x_val_sc, y_val_sc,                # В качестве параметров наши выборки
                    length = DEPTH,            # Анализируем по 21 прошедшим точкам
                    sampling_rate = 1,         # Для каждой точки
                    batch_size = BATCH_SIZE)   # Размер batch, который будем скармливать модели

print(train_datagen[0][0].shape,
      val_datagen[0][1].shape)

(64, 32, 5) (64, 5)


In [11]:
# Создадим генератор проверочной выборки, из которой потом вытащим xVal, yVal для проверки/оценки
DataGen = TSG(x_val_sc, y_val_sc,              # В качестве параметров наши выборки
              length = DEPTH,            # Анализируем по 21 прошедшим точкам
              sampling_rate = 1,         # Для каждой точки
              batch_size = len(x_val_sc))   # Размер batch будет равен длине нашей выборки


# сформируем сами массивы для проверки
XVAL, YVAL = [], []
for i in DataGen:
  XVAL.append(i[0])
  YVAL.append(i[1])

# так как это список объектов
# мы вытаскиваем нужный нам
XVAL = np.array(XVAL)[0]
YVAL = np.array(YVAL)[0]
print(XVAL.shape,YVAL.shape)

(530, 32, 50) (530, 5)


In [12]:
from scripts.make_models import baseline

baseline_pred5price= baseline(input_shape = XVAL.shape[1:],
                    pred_num = PREDICT_LAG,
                    activ_out = "linear",
                    depth= DEPTH)


# Отображение сводки модели
#model.summary()

# дерево модели
#tf.keras.utils.plot_model(model, show_shapes=True)

In [13]:
# понижение шага
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 mode='min',
                                                 factor = 0.8,
                                                 patience = 3,
                                                 min_lr = 1e-9,
                                                 verbose = 1)

# остановка
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

path_model = './predprice/'

# Сохранение модели
checkpoint  = tf.keras.callbacks.ModelCheckpoint(
                        filepath = path_model,
                        #save_weights_only=True,
                        monitor='val_loss',
                        mode='min',
                        save_best_only=True
                        )
# оптимизатор
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-2)

# функция потерь
loss = tf.keras.losses.Huber() #.MSE #
baseline_pred5price.compile(optimizer = optimizer, loss = loss, metrics=['mae', 'mape'])

In [14]:
# обучение модели вашей
history = baseline_pred5price.fit(train_datagen,                  # генератор данных для обучения
                    validation_data = val_datagen, # генератор данных для проверки
                    epochs = 2,
                    callbacks=[early_stopping, reduce_lr, checkpoint],
                    verbose = 1)

Epoch 1/2


: 